In [73]:
import requests
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [74]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

After loading the page and parsing it as html, the main table is queried thanks to its class which is `wikitable sortable`. In this table are queried all `tr`. The first `tr` that contains `th` is ignored by testings the number of `td` to be more than 0. The `text` of every third td is stipped with `.strip()`. `tr` with a postcode of value `'M5A'` are also ignored to explictely handle the case after browsing the table. The special case of the neighbourhood with a value of `'Not assigned'` is handled on the go.

In [87]:
data = []
for tr in soup.find('table', {'class' : 'wikitable sortable'}).find_all('tr'):
    tds = tr.find_all('td')
    if(len(tds) > 0 and tds[0].text != 'M5A' and tds[1].text != 'Not assigned'):
        # Handle neighbourhood == 'Not assigned'
        neighbourhood = tds[2].text.strip()
        if neighbourhood == 'Not assigned':
            neighbourhood = tds[1].text
        data.append([tds[0].text, tds[1].text, neighbourhood])
import pandas as pd
# Handle postcode == 'M5A'
data.append(['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'])
data = pd.DataFrame(data)
data.columns = ['Postcode',  'Borough', 'Neighbourhood']
data

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M6A,North York,Lawrence Heights
3,M6A,North York,Lawrence Manor
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
205,M8Z,Etobicoke,Mimico NW
206,M8Z,Etobicoke,The Queensway West
207,M8Z,Etobicoke,Royal York South West
208,M8Z,Etobicoke,South of Bloor


In [76]:
data.shape

(210, 3)

In [81]:
geo = pd.read_csv("week_3_Geospatial_Coordinates.csv") 
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [88]:
data = data.join(geo.set_index('Postal Code'), on='Postcode')
data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,Lawrence Heights,43.718518,-79.464763
3,M6A,North York,Lawrence Manor,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
205,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
206,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
207,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999
208,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999


As suggested, I work only with the data whom the borough contains "Toronto".

In [90]:
data = data[data['Borough'].str.contains("Toronto")]
data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
11,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
12,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
25,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
34,M4E,East Toronto,The Beaches,43.676357,-79.293031
...,...,...,...,...,...
189,M5X,Downtown Toronto,First Canadian Place,43.648429,-79.382280
190,M5X,Downtown Toronto,Underground city,43.648429,-79.382280
194,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
195,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [103]:
# create map of New York using latitude and longitude values
import folium 
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

The code above should display a dynamic version of the map bellow: ![map](week_3_map_toronto.jpg)